# Multi-class Classification
## One-vs-all Logisitic Regression

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio
import scipy.optimize as op
import copy

In [38]:
data = sio.loadmat("Practice\Machine Learning\machine-learning-ex3\ex3\ex3data1.mat")
data['X'] = np.insert(arr=data['X'], obj=0, values=1.0, axis=1)
X = data['X']
y = data['y']
m, n = X.shape

In [39]:
def sigmoid(Z):
    return 1/(1+ 1/(np.e**Z))

In [40]:
def costFunc(theta, X, y, lbd):
    m, n = X.shape
    y = y.reshape((m, 1))
    theta = theta.reshape((n, 1))
    Z = X.dot(theta) # M x 1
    H = sigmoid(Z)
    cost = 1/m*(-1 * y.T.dot(np.log(H)) - (1-y).T.dot(np.log(1-H)) )
    cost = cost + lbd/(2*m)*( np.sum(theta*theta)-theta[0]**2 )
    return cost

In [41]:
def gradFunc(theta, X, y, lbd):
    m, n = X.shape
    y = y.reshape((m, 1))
    theta = theta.reshape((n, 1))
    Z = X.dot(theta)
    H = sigmoid(Z) # M x 1
    grad = 1/m *( X.T.dot(H-y) + lbd*theta )
    grad[0] = grad[0] - lbd/m * theta[0]
    return grad

In [45]:
def oneVsAll(X, y, k, lmbda):
    # k classes
    m, n = X.shape
    theta = []
    for i in range(k):
            # iterate for i: [0:9]
            # mapping 1 -> theta[:, 0]
            # mapping 2 -> theta[:, 1]
            # ...
            # mapping 9 -> theta[:, 8]
            # mapping 0 -> theta[:, 9]
            yf = copy.deepcopy(y)
            yf[y!=i+1] = 0
            yf[y==i+1] = 1
            temp = op.minimize(
                fun=costFunc, x0=np.zeros((n, 1)),
                jac=gradFunc, args=(X, yf, lmbda), 
                method='TNC')
            theta.append(temp['x'])
    return np.array(theta).T

In [46]:
k = 10
theta = oneVsAll(X, y, 10, 1)

In [75]:
# predict
print(theta[0:5, 0:5])
Z = X.dot(theta)
H = sigmoid(Z)
p = np.where( H == np.amax(H, axis=1).reshape((m, 1)) )[1]
p = p+1
p = p.reshape((m, 1))


In [80]:
# Accuracy
acc= p[p == y].size/m*100
print(acc)